In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
from braindecode.datasets.bbci import BBCIDataset
from braindecode.mne_ext.signalproc import mne_apply
# we loaded all sensors to always get same cleaning results independent of sensor selection
# There is an inbuilt heuristic that tries to use only EEG channels and that definitely
# works for datasets in our paper
#train_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/13.mat')
#test_loader = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/test/13.mat')
start_cnt = BBCIDataset('/data/schirrmr/schirrmr/HGD-public/reduced/train/4.mat',).load()
start_cnt = start_cnt.drop_channels(['STI 014'])
def car(a):
    return a - np.mean(a, keepdims=True, axis=0)

start_cnt = mne_apply(
    car, start_cnt)

start_cnt = start_cnt.reorder_channels(['C3', 'C4'])

In [ ]:
from collections import OrderedDict
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

marker_def = OrderedDict([('Right Hand', [1]), ('Left Hand', [2],),
                         ('Rest', [3]), ('Feet', [4])])
ival = [500,1500]
from braindecode.mne_ext.signalproc import mne_apply, resample_cnt
from braindecode.datautil.signalproc import exponential_running_standardize, bandpass_cnt

log.info("Resampling train...")
cnt = resample_cnt(start_cnt, 250.0)
log.info("Standardizing train...")
cnt = mne_apply(lambda a: exponential_running_standardize(a.T ,factor_new=1e-3, init_block_size=1000, eps=1e-4).T,
                     cnt)
cnt = resample_cnt(cnt, 32.0)
cnt = resample_cnt(cnt, 64.0)
#cnt = mne_apply(
#    lambda a: bandpass_cnt(a, 0, 2, cnt.info['sfreq'],
#                           filt_order=10,
#                           axis=1), cnt)

train_set = create_signal_target_from_raw_mne(cnt, marker_def, ival)

In [ ]:
x_right = train_set.X[train_set.y == 0]

x_rest = train_set.X[train_set.y == 2]

inputs_a = np_to_var(x_right[:160,0:1,:,None], dtype=np.float32)

inputs_b = np_to_var(x_rest[:160,0:1,:,None], dtype=np.float32)
inputs = [inputs_a, inputs_b]

In [ ]:
cnt_bandpassed =  mne_apply(
    lambda a: bandpass_cnt(a, 8, 13, cnt.info['sfreq'],
                           filt_order=10,
                           axis=1), cnt)

In [ ]:
alpha_set = create_signal_target_from_raw_mne(cnt_bandpassed, marker_def, ival)
x_alpha_right = alpha_set.X[alpha_set.y == 0]

x_alpha_rest = alpha_set.X[alpha_set.y == 2]

alpha_a = np_to_var(x_alpha_right[:160,0:1,:,None], dtype=np.float32)

alpha_b = np_to_var(x_alpha_rest[:160,0:1,:,None], dtype=np.float32)
inputs_alpha = [alpha_a, alpha_b]

In [ ]:
from scipy.signal import hilbert
alpha_env_right = np.abs(hilbert(x_alpha_right))
alpha_env_rest = np.abs(hilbert(x_alpha_rest))

env_a = np_to_var(alpha_env_right[:160,0:1,:,None], dtype=np.float32)

env_b = np_to_var(alpha_env_rest[:160,0:1,:,None], dtype=np.float32)
inputs_env = [env_a, env_b]

In [ ]:
fig, axes = plt.subplots(80,4, figsize=(14,80), sharex=True, sharey=True)
for i_class in range(len(inputs)):
    for i_example in range(len(inputs_env[i_class])):
        i_row = i_example // 2
        i_col = i_example % 2
        i_col += i_class * 2
        axes[i_row][i_col].plot(var_to_np(inputs_env[i_class][i_example]).squeeze(),
                               color=seaborn.color_palette()[i_class])
fig.suptitle('Input signals', y=0.9)

from matplotlib.lines import Line2D
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
labels = ['Right', 'Rest',]
axes[0][-1].legend(lines, labels, bbox_to_anchor=(1,1,0,0))

In [ ]:
from hierarchical_gaussian import sample_hierarchically

In [ ]:
from reversible.gaussian import get_gauss_samples

def sample_hierarchically(n_samples, mean, log_stds):
    cur_mean = mean
    cur_log_std = log_std
    samples = th.zeros((n_samples, len(cur_mean)), dtype=th.float32)
    covs = th.zeros((len(cur_mean), len(cur_mean)), dtype=th.float32)
    for i_exp in range(int(np.log2(len(cur_mean))) + 1):
        cur_mean = th.stack(th.chunk(cur_mean, int(2**i_exp)))
        this_mean = th.mean(cur_mean, dim=1, keepdim=True)
        cur_mean = cur_mean - this_mean
        cur_mean = cur_mean.view(-1)
        this_log_std = log_stds[i_exp]
        # sample...
        this_samples = get_gauss_samples(n_samples, this_mean.squeeze(-1), th.exp(this_log_std).squeeze(-1))
        samples += this_samples.view(-1).repeat(
            len(cur_mean) // int(2**i_exp),1).t().contiguous().view(samples.shape)
        # compute cov matrix
        for i_part in range(2 ** i_exp):
            i_1, i_2 = int((i_part/2**i_exp) * len(covs)), int(((i_part+1)/2**i_exp) * len(covs))
            covs[i_1:i_2, i_1:i_2] += (th.exp(this_log_std[i_part]) ** 2)
    return samples, covs


In [ ]:
mean = means_per_cluster[i_class]
log_stds = stds_per_cluster[i_class]
samples, covs = sample_hierarchically(320, mean, log_stds)

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(2019011641, True)
n_clusters = len(inputs)
n_dims = inputs[0].shape[2]
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]


# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [[th.zeros(2 ** i_exp, requires_grad=True) for i_exp in range(int(np.log2(len(mean))) + 1)]
                    for _ in range(n_clusters)]

import itertools



optimizer = th.optim.Adam(means_per_cluster + list(itertools.chain(*stds_per_cluster)),
                          lr=1e-2)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    i_class = 0
    this_ins = inputs_env[i_class].squeeze()
    mean = means_per_cluster[i_class]
    log_stds = stds_per_cluster[i_class]
    samples, covs = sample_hierarchically(2*320, mean, log_stds)
    sliced_loss = sliced_from_samples(this_ins, samples, 10, None)
    optimizer.zero_grad()
    sliced_loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        display_text("Sliced loss {:.2f}".format(sliced_loss))
        denominator = th.mm(th.sqrt(th.diag(covs)).unsqueeze(1), th.sqrt(th.diag(covs)).unsqueeze(0))
        corrs = covs / denominator
        corrs = var_to_np(corrs)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(corrs, vmin=-1, vmax=1, cmap=cm.coolwarm)
        cbar = plt.colorbar()
        cbar.set_label("Correlation")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(var_to_np(covs), cmap=cm.Reds, vmin=0)
        cbar = plt.colorbar()
        cbar.set_label("Covariance")
        display(fig)
        plt.close(fig)
        
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(samples[:3]).T)
        display(fig)
        plt.close(fig)

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(2019011641, True)
n_clusters = len(inputs)
n_dims = inputs[0].shape[2] // 2
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]


# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [[th.zeros(2 ** i_exp, requires_grad=True) for i_exp in range(int(np.log2(len(mean))) + 1)]
                    for _ in range(n_clusters)]

import itertools



optimizer = th.optim.Adam(means_per_cluster + list(itertools.chain(*stds_per_cluster)),
                          lr=1e-2)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    i_class = 0
    this_ins = inputs_env[i_class].squeeze()
    mean = means_per_cluster[i_class]
    log_stds = stds_per_cluster[i_class]
    samples, covs = sample_hierarchically(2*320, mean, log_stds)
    samples = nn.functional.interpolate(samples.unsqueeze(1).unsqueeze(3),size=(64,1),
                                        mode='bilinear', align_corners=False).squeeze()
    sliced_loss = sliced_from_samples(this_ins, samples, 10, None)
    optimizer.zero_grad()
    sliced_loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        display_text("Sliced loss {:.2f}".format(sliced_loss))
        denominator = th.mm(th.sqrt(th.diag(covs)).unsqueeze(1), th.sqrt(th.diag(covs)).unsqueeze(0))
        corrs = covs / denominator
        corrs = var_to_np(corrs)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(corrs, vmin=-1, vmax=1, cmap=cm.coolwarm)
        cbar = plt.colorbar()
        cbar.set_label("Correlation")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(var_to_np(covs), cmap=cm.Reds, vmin=0)
        cbar = plt.colorbar()
        cbar.set_label("Covariance")
        display(fig)
        plt.close(fig)
        
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(samples[:3]).T)
        display(fig)
        plt.close(fig)

### add alpha rhythm

In [ ]:
def create_in_samples(n_samples):
    samples, covs = sample_hierarchically(n_samples, mean, log_stds)
    interpolated = nn.functional.interpolate(samples.unsqueeze(1).unsqueeze(3),size=(64,1),
                                        mode='bilinear', align_corners=False).squeeze()

    phase = th.rand(n_samples) * 2 * np.pi
    step = 64 / (2*np.pi * freq)
    timecourses = interpolated
    template = th.sin((th.linspace(0,63,64) * step).unsqueeze(0) + phase.unsqueeze(1))
    examples = template  * timecourses 
    return examples, covs

In [ ]:
from reversible.util import set_random_seeds
set_random_seeds(2019011641, True)
n_clusters = len(inputs)
n_dims = inputs[0].shape[2] // 2
means_per_cluster = [th.autograd.Variable(th.ones(n_dims), requires_grad=True)
                     for _ in range(n_clusters)]


# keep in mind this is in log domain so 0 is std 1
stds_per_cluster = [[th.zeros(2 ** i_exp, requires_grad=True) for i_exp in range(int(np.log2(len(mean))) + 1)]
                    for _ in range(n_clusters)]

import itertools


freq = th.ones(1, dtype=th.float32, requires_grad=True)
freq.data[0] = 10

optimizer = th.optim.Adam(means_per_cluster + list(itertools.chain(*stds_per_cluster)) + [freq],
                          lr=1e-2)

In [ ]:
n_epochs = 2000
for i_epoch in range(n_epochs):
    i_class = 0
    this_ins = inputs_alpha[i_class].squeeze()
    mean = means_per_cluster[i_class]
    log_stds = stds_per_cluster[i_class]
    n_samples = 2*320
    examples, covs = create_in_samples(n_samples)
    sliced_loss = sliced_from_samples(this_ins, examples, 10, None)
    optimizer.zero_grad()
    sliced_loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        display_text("Sliced loss {:.2f}".format(sliced_loss))
        denominator = th.mm(th.sqrt(th.diag(covs)).unsqueeze(1), th.sqrt(th.diag(covs)).unsqueeze(0))
        corrs = covs / denominator
        corrs = var_to_np(corrs)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(corrs, vmin=-1, vmax=1, cmap=cm.coolwarm)
        cbar = plt.colorbar()
        cbar.set_label("Correlation")
        display(fig)
        plt.close(fig)
        fig = plt.figure(figsize=(3,3))
        plt.imshow(var_to_np(covs), cmap=cm.Reds, vmin=0)
        cbar = plt.colorbar()
        cbar.set_label("Covariance")
        display(fig)
        plt.close(fig)
        
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(examples[:3]).T)
        display(fig)
        plt.close(fig)
        n_samples = 3200
        in_samples = var_to_np(create_in_samples(n_samples)[0]).squeeze()

        this_ins = var_to_np(inputs_alpha[i_class].squeeze())

        diffs = this_ins[:,None] - in_samples[None]

        diffs = np.sqrt(np.sum(np.square(diffs), axis=-1))

        import ot
        coupling = ot.emd([],[], diffs)
        mask = coupling > (1/(2*len(in_samples)))
        assert np.sum(mask)  == len(in_samples)
        argmaxes = np.argmax(mask, axis=0)


        fig, axes = plt.subplots(10,4, figsize=(16,20), sharex=True, sharey=True)
        for i_in, ax in enumerate(axes.flatten()):
            i_samples = np.nonzero(argmaxes == i_in)
            ax.plot(in_samples[i_samples].T, color=seaborn.color_palette()[0], lw=0.5)
            ax.plot(this_ins[i_in], color='black')
        display(fig)
        plt.close(fig)
        display([th.exp(s) for s in log_stds])


In [ ]:
np.min(np.cov(var_to_np(inputs_env[0].squeeze()).T))

In [ ]:
np.max(np.cov(var_to_np(inputs_env[0].squeeze()).T))

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(np.cov(var_to_np(inputs_env[0].squeeze()).T), vmin=-0.7, vmax=0.7,
           cmap=cm.coolwarm)

cbar = plt.colorbar()
cbar.set_label("Covariance")
cbar.set_ticks([0,0.5])


In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(np.corrcoef(var_to_np(inputs_env[0].squeeze()).T), vmin=-1, vmax=1,
           cmap=cm.coolwarm)

In [ ]:
var_to_np(this_std).repeat(2)

In [ ]:
var_to_np(inds)

In [ ]:

from matplotlib import rcParams, cycler
cmap = plt.cm.copper
N = int(np.log2(len(mean))) + 1
plt.figure(figsize=(8,5))
with plt.rc_context({'axes.prop_cycle': cycler(color=cmap(np.linspace(0., 1, N)))}):
    with seaborn.axes_style("whitegrid"):
        for i_exp in range(int(np.log2(len(mean))) + 1):
            this_std = th.exp(log_stds[i_exp])
            inds = np.arange(0,32,32 / 2**i_exp)
            inds = inds.repeat(2)
            inds[0::2] -= 0.1
            inds = np.append(inds, 32)
            inds = inds[1:]
            plt.plot(inds, var_to_np(this_std).repeat(2), lw=3-i_exp*0.4)


In [ ]:
th.exp(log_stds[i_exp])

In [ ]:
plt.plot(var_to_np(mean))
plt.plot(np.mean(var_to_np(inputs_env[0]), axis=0).squeeze()[::2])

In [ ]:
fig = plt.figure(figsize=(12,4))
plt.plot(var_to_np(this_ins[:10]).T, color=seaborn.color_palette()[0], lw=0.5)
plt.plot(var_to_np(samples[:10]).T, color=seaborn.color_palette()[1], lw=0.5)
display(fig)
plt.close(fig)

In [ ]:
plt.plot(var_to_np(this_ins)[:3].T)

In [ ]:
plt.plot(var_to_np(mean))
plt.plot(np.mean(var_to_np(this_ins), axis=0))

In [ ]:
cur_mean = th.randn(64)#np_to_var([0,2,0,5], dtype=np.float32)
cur_log_std = th.randn(64)#np_to_var([2,3,1,0.5], dtype=np.float32)
n_samples = 2000
samples, covs = sample_hierarchically(n_samples, cur_mean, cur_log_std)

In [ ]:
plt.imshow(var_to_np(covs), cmap=cm.Reds)

In [ ]:
denominator = th.mm(th.sqrt(th.diag(covs)).unsqueeze(1), th.sqrt(th.diag(covs)).unsqueeze(0))

corrs = covs / denominator
plt.figure(figsize=(5,5))
plt.imshow(corrs, vmin=-1, vmax=1, cmap=cm.coolwarm)

In [ ]:
%%time
cur_mean = th.randn(64)#np_to_var([0,2,0,5], dtype=np.float32)
cur_log_std = th.randn(64)#np_to_var([2,3,1,0.5], dtype=np.float32)
n_samples = 2000
samples = th.zeros((n_samples, len(cur_mean)), dtype=th.float32)
covs = th.zeros((len(cur_mean), len(cur_mean)), dtype=th.float32)

for i_exp in range(int(np.log2(len(cur_mean))) + 1):
    cur_mean = th.stack(th.chunk(cur_mean, int(2**i_exp)))
    this_mean = th.mean(cur_mean, dim=1, keepdim=True)
    cur_mean = cur_mean - this_mean
    cur_mean = cur_mean.view(-1)
    cur_log_std = th.stack(th.chunk(cur_log_std, int(2**i_exp)))
    this_log_std = th.mean(cur_log_std, dim=1, keepdim=True)
    cur_log_std = cur_log_std - this_log_std
    cur_log_std = cur_log_std.view(-1)
    # sample...
    this_samples = get_gauss_samples(n_samples, this_mean.squeeze(-1), th.exp(this_log_std).squeeze(-1))
    samples += this_samples.view(-1).repeat(
        len(cur_mean) // int(2**i_exp),1).t().contiguous().view(samples.shape)
    # compute cov matrix
    for i_part in range(2 ** i_exp):
        i_1, i_2 = int((i_part/2**i_exp) * len(covs)), int(((i_part+1)/2**i_exp) * len(covs))
        covs[i_1:i_2, i_1:i_2] += (th.exp(this_log_std.squeeze(-1)[i_part]) ** 2)


In [ ]:
plt.imshow(var_to_np(covs), cmap=cm.Reds)

In [ ]:
covs

In [ ]:
denominator = th.mm(th.sqrt(th.diag(covs)).unsqueeze(1), th.sqrt(th.diag(covs)).unsqueeze(0))

corrs = covs / denominator
plt.figure(figsize=(5,5))
plt.imshow(corrs, vmin=-1, vmax=1, cmap=cm.coolwarm)

In [ ]:
th.min(corrs)

In [ ]:
corrs

In [ ]:
emp_covs = np.cov(var_to_np(samples).T)

In [ ]:
emp_covs

In [ ]:
th.mean(samples, dim=0)

In [ ]:
covs = np.cov(var_to_np(samples).T)

In [ ]:
covs

In [ ]:
plt.imshow(covs, vmin=0, vmax=3, cmap=cm.Reds)

In [ ]:
summed

In [ ]:
cur_mean = means_per_cluster[0]
i_exp = 0
for i_exp in range(int(np.log2(len(mean)))):
    cur_mean = th.stack(th.chunk(cur_mean, int(2**i_exp)))

    this_mean = th.mean(cur_mean, dim=1, keepdim=True)
    print(this_mean)
    # sample...
    cur_mean = cur_mean - this_mean
    print(cur_mean.shape)
    cur_mean = cur_mean.view(-1)




In [ ]:
this_samples = get_gauss_samples(3, th.mean(this_mean, dim=1), th.exp(th.mean(this_log_std, dim=1)), truncate_to=3)

this_samples.repeat((1,64))
